In [1]:
from __future__ import division
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import time
import argparse
import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import torch.nn.functional as F
from utils import accuracy, AverageMeter, save_checkpoint, visualize_graph, get_parameters_size
from torch.utils.tensorboard import SummaryWriter
from net_factory import get_network_fn


# dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import DataParallel
import tqdm
import numpy as np
# del test_loader
import tqdm
# test_loader

In [2]:

parser = argparse.ArgumentParser(description='PyTorch GCN MNIST Training')

parser.add_argument('--epochs', default=50, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=128, type=int,
                    metavar='N', help='mini-batch size (default: 64)')
parser.add_argument('--lr', '--learning-rate', default=0.01, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    metavar='N', help='print frequency (default: 10)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--pretrained', default='', type=str, metavar='PATH',
                    help='path to pretrained checkpoint (default: none)')
parser.add_argument('--gpu', default=0, type=int,
                    metavar='N', help='GPU device ID (default: -1)')
parser.add_argument('--dataset_dir', default='../../MNIST', type=str, metavar='PATH',
                    help='path to dataset (default: ../MNIST)')
parser.add_argument('--comment', default='', type=str, metavar='INFO',
                    help='Extra description for tensorboard')
parser.add_argument('--model', default='gcn', type=str, metavar='NETWORK',
                    help='Network to train')

args = parser.parse_args(args=[])

use_cuda = (args.gpu >= 0) and torch.cuda.is_available()
best_prec1 = 0
writer = SummaryWriter(comment='_'+args.model+'_'+args.comment)
iteration = 0

# # from loaddataset import load_data
# from loaddataset import load_data

# batch_size = 64
# train_loader = DataLoader(load_data(training=True), batch_size=batch_size, shuffle=True)  # ,prefetch_factor=2
# test_loader = DataLoader(load_data(training=False), batch_size=batch_size, shuffle=True)  # ,prefetch_factor=2


In [3]:

# Load model
model = get_network_fn(name='GCN2wayHashingsimple2')#GCNCNN
# print(model)

# Try to visulize the model
try:
	visualize_graph(model, writer, input_size=(1, 5, 128, 128))
except:
	print('\nNetwork Visualization Failed! But the training procedure continue.')


/root/autodl-nas/Gabor_CNN_PyTorch/gcn/layers/GConv.py:67: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for i in range(x.size(1)):
/root/autodl-nas/Gabor_CNN_PyTorch/demo/net_factory.py:194: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return x + self.pe[:, :, :x.size(2), :x.size(3)]


In [4]:
# out = model(torch.rand(5, 5, 128, 128))
# print(out.size())

In [5]:
# # Calculate the total parameters of the model
# print('Model size: {:0.2f} million float parameters'.format(get_parameters_size(model)/1e6))
# args.pretrained = 'model_best_gcn2waycqhashing.pth.tar'
# if os.path.isfile(args.pretrained):
#     print("=> loading checkpoint '{}'".format(args.pretrained))
#     checkpoint = torch.load(args.pretrained,map_location=torch.device('cpu'))
#     model.load_state_dict(checkpoint['state_dict'])
# else:
#     print("=> no checkpoint found at '{}'".format(args.pretrained))
# print(checkpoint['best_prec1'])

In [6]:
class CSQLoss(torch.nn.Module):
    def __init__(self, config, bit):
        super(CSQLoss, self).__init__()
        self.is_single_label = config["dataset"] not in {"nuswide_21", "nuswide_21_m", "coco"}
        self.hash_targets = self.get_hash_targets(config["n_class"], bit).to(config["device"])
        self.multi_label_random_center = torch.randint(2, (bit,)).float().to(config["device"])
        self.criterion = torch.nn.BCELoss().to(config["device"])

    def forward(self, u, y, ind, config):
        u = u.tanh()
        hash_center = self.label2center(y)
        center_loss = self.criterion(0.5 * (u + 1), 0.5 * (hash_center + 1))

        Q_loss = (u.abs() - 1).pow(2).mean()
        return center_loss + config["lambda"] * Q_loss

    def label2center(self, y):
        if self.is_single_label:
            hash_center = self.hash_targets[y.argmax(axis=1)]
        else:
            # to get sign no need to use mean, use sum here
            center_sum = y @ self.hash_targets
            random_center = self.multi_label_random_center.repeat(center_sum.shape[0], 1)
            center_sum[center_sum == 0] = random_center[center_sum == 0]
            hash_center = 2 * (center_sum > 0).float() - 1
        return hash_center

    # use algorithm 1 to generate hash centers
    def get_hash_targets(self, n_class, bit):
        H_K = hadamard(bit)
        H_2K = np.concatenate((H_K, -H_K), 0)
        hash_targets = torch.from_numpy(H_2K[:n_class]).float()

        if H_2K.shape[0] < n_class:
            hash_targets.resize_(n_class, bit)
            for k in range(20):
                for index in range(H_2K.shape[0], n_class):
                    ones = torch.ones(bit)
                    # Bernouli distribution
                    sa = random.sample(list(range(bit)), bit // 2)
                    ones[sa] = -1
                    hash_targets[index] = ones
                # to find average/min  pairwise distance
                c = []
                for i in range(n_class):
                    for j in range(n_class):
                        if i < j:
                            TF = sum(hash_targets[i] != hash_targets[j])
                            c.append(TF)
                c = np.array(c)

                # choose min(c) in the range of K/4 to K/3
                # see in https://github.com/yuanli2333/Hadamard-Matrix-for-hashing/issues/1
                # but it is hard when bit is  small
                if c.min() > bit / 4 and c.mean() >= bit / 2:
                    print(c.min(), c.mean())
                    break
        return hash_targets



In [7]:
# from torchvision import datasets, models, transforms

# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model.fc = nn.Linear(num_ftrs, 450)


In [8]:
import numpy as np
import os
# import cv2
from  matplotlib import pyplot as plt

## torch
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchsummary import summary

# dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import DataParallel
import tqdm


# read image
import PIL
from PIL import Image
from utils import *

ms_polyu_path = 'dataset/MS_PolyU/'
casia_path = 'dataset/CASIA-Multi-Spectral-PalmprintV1/images/'

r_img_path = ms_polyu_path + 'Red_ind/'
b_img_path =  ms_polyu_path + 'Blue_ind/'
n_img_path =  ms_polyu_path + 'NIR_ind/'
g_img_path =  ms_polyu_path + 'Green_ind/'

################ DATASET CLASS
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 
# one_hot_embedding(1, 10)
def part_init(istrain=True):
    r_list = []
    b_list = []
    vein_list = []
    prints_list = []
    labels = []
    
        # split all data into train, test data
    train_ratio = 1
    train_num = int(500 * train_ratio)
    print("split train users:",train_num)
    if istrain:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
                g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

                b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)

                imgprint = np.dstack((r_img,g_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)
    else:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8,12):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
                g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

                b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)
                imgprint = np.dstack((r_img,g_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)



    # return np.array(r_list), np.array(b_list), np.array(n_list), np.array(labels),np.array(r_list_test), np.array(b_list_test), np.array(n_list_test), np.array(labels_test)
    return  vein_list,prints_list, labels

# r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
class load_data(Dataset):
    """Loads the Data."""
    def __init__(self, training=True):

        self.training = training
#         r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
        self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ColorJitter(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomPerspective(),
        transforms.RandomAffine(30),
        transforms.ToTensor(),
#         transforms.Resize((224, 224)),# if resnet
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        self.transform_test = transforms.Compose([
        transforms.ToPILImage(),
#         transforms.Resize((224, 224)),# if resnet
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        if self.training:
            print('\n...... Train files loading\n')
            self.vein_list,self.prints_list, self.labels= part_init(istrain=True)
            print('\nTrain files loaded ......\n')
        else:
            print('\n...... Test files loading\n')
            self.vein_list,self.prints_list, self.labels = part_init(istrain=False)
            print('\nTest files loaded ......\n')

    def __len__(self):
        return len(self.vein_list)

         
    def __getitem__(self, idx):

        if self.training:
            prints_img = self.transform(self.prints_list[idx])
            vein_img = self.transform(self.vein_list[idx])
        else:
            prints_img = self.transform_test(self.prints_list[idx])
            vein_img = self.transform_test(self.vein_list[idx])
        
        label = self.labels[idx]
        
        n_img = np.dstack((prints_img[0,:,:],prints_img[1,:,:],prints_img[2,:,:],vein_img[0,:,:],vein_img[1,:,:]))
        
        return n_img,label


In [9]:
def get_config():
    config = {
        "lambda": 0.1,
        "optimizer": {"type": optim.RMSprop, "optim_params": {"lr": 1e-5, "weight_decay": 10 ** -5}},
        "info": "[CSQ]",
        "resize_size": 256,
        "crop_size": 224,
        "batch_size": 64,
        # "net": AlexNet,
        "net": "w",
        "dataset": "palmprint",
        "n_class":500,
        # "dataset": "imagenet",
        # "dataset": "coco",
        # "dataset": "nuswide_21",
        # "dataset": "nuswide_21_m",
        "epoch": 500,
        "test_map": 10,
        # "device":torch.device("cpu"),
        "device": torch.device("cuda:0"),
        "bit_list": [1024],
    }
#     config = config_dataset(config)
    return config



config = get_config()
print(config)
bit = 1024

device = torch.device("cuda:0")


{'lambda': 0.1, 'optimizer': {'type': <class 'torch.optim.rmsprop.RMSprop'>, 'optim_params': {'lr': 1e-05, 'weight_decay': 1e-05}}, 'info': '[CSQ]', 'resize_size': 256, 'crop_size': 224, 'batch_size': 64, 'net': 'w', 'dataset': 'palmprint', 'n_class': 500, 'epoch': 500, 'test_map': 10, 'device': device(type='cuda', index=0), 'bit_list': [1024]}


In [10]:
# train_loader, test_loader, dataset_loader, num_train, num_test, num_dataset = get_data(config)
batch_size = 300
train_loader = DataLoader(load_data(training=True), batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,prefetch_factor=8)  # ,prefetch_factor=2
test_loader = DataLoader(load_data(training=False), batch_size=64, shuffle=False)  # ,prefetch_factor=2



...... Train files loading

split train users: 500


100%|██████████| 500/500 [01:23<00:00,  6.02it/s]



Train files loaded ......


...... Test files loading

split train users: 500


100%|██████████| 500/500 [00:43<00:00, 11.53it/s]


Test files loaded ......



In [11]:

# batch_size = 32
dataset_loader = train_loader
num_train = 4000
num_test = 2000
num_dataset = 6000



In [12]:
# !pip install scipy

In [13]:
from tools import *
# from network import *

import os
import torch
import torch.optim as optim
import time
import numpy as np
from scipy.linalg import hadamard  # direct import  hadamrd matrix from scipy
import random

torch.multiprocessing.set_sharing_strategy('file_system')


class CSQLoss(torch.nn.Module):
    def __init__(self, config, bit):
        super(CSQLoss, self).__init__()
        self.is_single_label = config["dataset"] not in {"nuswide_21", "nuswide_21_m", "coco"}
        self.hash_targets = self.get_hash_targets(config["n_class"], bit).to(config["device"])
        self.multi_label_random_center = torch.randint(2, (bit,)).float().to(config["device"])
        self.criterion = torch.nn.BCELoss().to(config["device"])

    def forward(self, u, y, ind, config):
        u = u.tanh()
        hash_center = self.label2center(y)
        center_loss = self.criterion(0.5 * (u + 1), 0.5 * (hash_center + 1))

        Q_loss = (u.abs() - 1).pow(2).mean()
        return center_loss + config["lambda"] * Q_loss

    def label2center(self, y):
        if self.is_single_label:
            hash_center = self.hash_targets[y.argmax(axis=1)]
        else:
            # to get sign no need to use mean, use sum here
            center_sum = y @ self.hash_targets
            random_center = self.multi_label_random_center.repeat(center_sum.shape[0], 1)
            center_sum[center_sum == 0] = random_center[center_sum == 0]
            hash_center = 2 * (center_sum > 0).float() - 1
        return hash_center

    # use algorithm 1 to generate hash centers
    def get_hash_targets(self, n_class, bit):
        H_K = hadamard(bit)
        H_2K = np.concatenate((H_K, -H_K), 0)
        hash_targets = torch.from_numpy(H_2K[:n_class]).float()

        if H_2K.shape[0] < n_class:
            hash_targets.resize_(n_class, bit)
            for k in range(20):
                for index in range(H_2K.shape[0], n_class):
                    ones = torch.ones(bit)
                    # Bernouli distribution
                    sa = random.sample(list(range(bit)), bit // 2)
                    ones[sa] = -1
                    hash_targets[index] = ones
                # to find average/min  pairwise distance
                c = []
                for i in range(n_class):
                    for j in range(n_class):
                        if i < j:
                            TF = sum(hash_targets[i] != hash_targets[j])
                            c.append(TF)
                c = np.array(c)

                # choose min(c) in the range of K/4 to K/3
                # see in https://github.com/yuanli2333/Hadamard-Matrix-for-hashing/issues/1
                # but it is hard when bit is  small
                if c.min() > bit / 4 and c.mean() >= bit / 2:
                    print(c.min(), c.mean())
                    break
        return hash_targets



In [14]:
# # Calculate the total parameters of the model
# print('Model size: {:0.2f} million float parameters'.format(get_parameters_size(model)/1e6))
# args.pretrained = 'model_best.pth.tar'
# if os.path.isfile(args.pretrained):
#     print("=> loading checkpoint '{}'".format(args.pretrained))
#     checkpoint = torch.load(args.pretrained)
#     model.load_state_dict(checkpoint['state_dict'])
# else:
#     print("=> no checkpoint found at '{}'".format(args.pretrained))


In [15]:
config["num_train"] = num_train
model = model.to(device)

optimizer = config["optimizer"]["type"](model.parameters(), **(config["optimizer"]["optim_params"]))

criterion = CSQLoss(config, bit)

# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=args.momentum, weight_decay=3e-05)
scheduler = StepLR(optimizer, step_size=100, gamma=0.9)


In [16]:
# device

In [17]:
import numpy as np

def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

A = np.random.randn(1,2)*10
# print(A)
# print(normalized(A,0))
print(normalized(A,1))# ok verified


def test(net,test_loader):
    test_loss = AverageMeter()
    acc = AverageMeter()
    FEATS = []
    GT = []
    net.eval()
    features = {}
    with torch.no_grad():
        for batch_idx, img in enumerate(test_loader):
            rbn = img[0].permute(0, 3, 1, 2).to(device, dtype=torch.float)
            label = img[1].to(device)

            output = net(rbn)
            FEATS.append(output.cpu().numpy())
#             FEATS.append(features['feats'].cpu().numpy())
            GT.append(img[1].numpy())

    GCNFEATS = np.concatenate(FEATS)
    GT = np.concatenate(GT)
    GCNFEATS = normalized(GCNFEATS,1)
    print('- feats shape:', GCNFEATS.shape)
    print('- GT shape:', GT.shape)
    from numpy import dot
    from numpy.linalg import norm

    def cossim(a,b):
        return dot(a, b)/(norm(a)*norm(b))

    pred_scores = []
    gt_label = []

    for i in range(2000):
        for j in range(i+1,2000):
            # pred_scores.append(final[i,j].detach().cpu().numpy())
            a = cossim(GCNFEATS[i,:],GCNFEATS[j,:])
            pred_scores.append(a)
            gt_label.append(i//4 == j//4)

    pred_scores = np.array(pred_scores)
    gt_label = np.array(gt_label)

    Gen = pred_scores[gt_label]
    Imp = pred_scores[gt_label==False]
    Imp = Imp[np.random.permutation(len(Imp))[:len(Gen)]]


#     import seaborn as sns
#     sns.distplot(Gen,  kde=False, label='Gen')
#     # df =gapminder[gapminder.continent == 'Americas']
#     sns.distplot(Imp,  kde=False,label='Imp')
#     # Plot formatting
#     plt.legend(prop={'size': 12})
#     plt.title('Life Expectancy of Two Continents')
#     plt.xlabel('Life Exp (years)')
#     plt.ylabel('Density')

    from pyeer.eer_info import get_eer_stats
    from pyeer.report import generate_eer_report, export_error_rates
    from pyeer.plot import plot_eer_stats


    # Calculating stats for classifier A
    stats_a = get_eer_stats(Gen, Imp)
    print(stats_a.eer)

    return stats_a.eer

##### INSPECT FEATURES



[[ 0.72978753 -0.68367402]]


In [ ]:
Best_eer = 1.0

config["epoch"] = 2000
for epoch in range(500,config["epoch"]):

    current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

    print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
        config["info"], epoch + 1, config["epoch"], current_time, bit, config["dataset"]), end="")

    model.train()

    train_loss = 0
    for batch_idx, img in enumerate(train_loader):
        image = img[0].permute(0, 3, 1, 2).to(device, dtype=torch.float)
        label = img[1].to(device)

        optimizer.zero_grad()
        u = model(image)

        loss = criterion(u, label.float(), 0, config)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(train_loader)
    if epoch % 50 ==0:
        eer = test(model,test_loader)
        if eer < Best_eer:
            Best_eer = eer
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': Best_eer,
                'optimizer' : optimizer.state_dict(),
            }, True, filename='checkpointGCN2wayHashingsimple.pth.tar', remark='GCN2wayHashingsimple')
        print("%s epoch:%d, bit:%d, dataset:%s,eer:%.5f, Best eer: %.5f" % (
            config["info"], epoch + 1, bit, config["dataset"], eer, Best_eer))
    print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))##loss:0.625
    scheduler.step()

#     if (epoch + 1) % config["test_map"] == 0:
#         # print("calculating test binary code......")
#         tst_binary, tst_label = compute_result(test_loader, net, device=device)

#         # print("calculating dataset binary code.......")\
#         trn_binary, trn_label = compute_result(dataset_loader, net, device=device)

#         # print("calculating map.......")
#         mAP = CalcTopMap(trn_binary.numpy(), tst_binary.numpy(), trn_label.numpy(), tst_label.numpy(),
#                          config["topK"])

#         if mAP > Best_mAP:
#             Best_mAP = mAP

#             if "save_path" in config:
#                 if not os.path.exists(config["save_path"]):
#                     os.makedirs(config["save_path"])
#                 print("save in ", config["save_path"])
#                 np.save(os.path.join(config["save_path"], config["dataset"] + str(mAP) + "-" + "trn_binary.npy"),
#                         trn_binary.numpy())
#                 torch.save(net.state_dict(),
#                            os.path.join(config["save_path"], config["dataset"] + "-" + str(mAP) + "-model.pt"))
#         print("%s epoch:%d, bit:%d, dataset:%s, MAP:%.3f, Best MAP: %.3f" % (
#             config["info"], epoch + 1, bit, config["dataset"], mAP, Best_mAP))
#         print(config)


[CSQ][501/2000][13:01:37] bit:1024, dataset:palmprint, training....- feats shape: (2000, 1024)
- GT shape: (2000, 500)
0.02266666666666667
[CSQ] epoch:501, bit:1024, dataset:palmprint,eer:0.02267, Best eer: 0.02267
 loss:1.34724
[CSQ][502/2000][13:03:23] bit:1024, dataset:palmprint, training... loss:1.13188
[CSQ][503/2000][13:03:36] bit:1024, dataset:palmprint, training... loss:1.08966
[CSQ][504/2000][13:03:49] bit:1024, dataset:palmprint, training... loss:1.05679
[CSQ][505/2000][13:04:04] bit:1024, dataset:palmprint, training... loss:1.03082
[CSQ][506/2000][13:04:16] bit:1024, dataset:palmprint, training... loss:1.00820
[CSQ][507/2000][13:04:30] bit:1024, dataset:palmprint, training... loss:0.98915
[CSQ][508/2000][13:04:43] bit:1024, dataset:palmprint, training... loss:0.97001
[CSQ][509/2000][13:04:56] bit:1024, dataset:palmprint, training... loss:0.95684
[CSQ][510/2000][13:05:11] bit:1024, dataset:palmprint, training... loss:0.94163
[CSQ][511/2000][13:05:24] bit:1024, dataset:palmpr

In [ ]:
# # if epoch % 50 ==0:
# save_checkpoint({
#     'epoch': epoch + 1,
#     'state_dict': net.state_dict(),
#     'best_prec1': 0,
#     'optimizer' : optimizer.state_dict(),
# }, True)

In [ ]:
# for _, img in tqdm.tqdm(enumerate(train_loader)):
#     print((img[0].size(), img[1].size()))
#     break
    
# for _, img in tqdm.tqdm(enumerate(test_loader)):
#     print((img[0].size(), img[1].size()))
#     break

In [ ]:
# print(img[0].permute(0, 3, 1, 2).size())
# print(img[0].permute(0, 3, 1, 2).double().dtype )

In [ ]:
model = model.float()

In [ ]:
# outs = model(img[0].permute(0, 3, 1, 2).float())
# print(outs.size())

In [ ]:
# # Calculate the total parameters of the model
# print('Model size: {:0.2f} million float parameters'.format(get_parameters_size(model)/1e6))
# args.pretrained = 'model_best_gcn2wayca.pth.tar'
# if os.path.isfile(args.pretrained):
#     print("=> loading checkpoint '{}'".format(args.pretrained))
#     checkpoint = torch.load(args.pretrained)
#     model.load_state_dict(checkpoint['state_dict'])
# else:
#     print("=> no checkpoint found at '{}'".format(args.pretrained))


In [ ]:
test_loader = DataLoader(load_data(training=False), batch_size=batch_size, shuffle=False)  # ,prefetch_factor=2

In [ ]:
for _, img in tqdm.tqdm(enumerate(test_loader)):
    print((img[0].size(), img[1].size()))
    break
print(img[1])

In [ ]:
# ##### HELPER FUNCTION FOR FEATURE EXTRACTION
# # print(model)

# def get_features(name):
#     def hook(model, input, output):
#         features[name] = output.detach()
#     return hook
# ##### REGISTER HOOK
# model.model_print[16].register_forward_hook(get_features('feats'))
# # model.fc1.register_forward_hook(get_features('feats'))

In [ ]:

def test(net,test_loader):
    test_loss = AverageMeter()
    acc = AverageMeter()
    FEATS = []
    GT = []
    features = {}
    with torch.no_grad():
        for batch_idx, img in enumerate(test_loader):
            rbn = img[0].permute(0, 3, 1, 2).to(device, dtype=torch.float)
            label = img[1].to(device)

            output = net(rbn)
            FEATS.append(output.cpu().numpy())
#             FEATS.append(features['feats'].cpu().numpy())
            GT.append(img[1].numpy())

    GCNFEATS = np.concatenate(FEATS)
    GT = np.concatenate(GT)
    print('- feats shape:', GCNFEATS.shape)
    print('- GT shape:', GT.shape)
    from numpy import dot
    from numpy.linalg import norm

    def cossim(a,b):
        return dot(a, b)/(norm(a)*norm(b))

    pred_scores = []
    gt_label = []

    for i in tqdm.tqdm(range(3000)):
        for j in range(i+1,3000):
            # pred_scores.append(final[i,j].detach().cpu().numpy())
            a = cossim(GCNFEATS[i,:],GCNFEATS[j,:])
            pred_scores.append(a)
            gt_label.append(i//6 == j//6)

    pred_scores = np.array(pred_scores)
    gt_label = np.array(gt_label)

    Gen = pred_scores[gt_label]
    Imp = pred_scores[gt_label==False]
    Imp = Imp[np.random.permutation(len(Imp))[:len(Gen)]]


#     import seaborn as sns
#     sns.distplot(Gen,  kde=False, label='Gen')
#     # df =gapminder[gapminder.continent == 'Americas']
#     sns.distplot(Imp,  kde=False,label='Imp')
#     # Plot formatting
#     plt.legend(prop={'size': 12})
#     plt.title('Life Expectancy of Two Continents')
#     plt.xlabel('Life Exp (years)')
#     plt.ylabel('Density')

    from pyeer.eer_info import get_eer_stats
    from pyeer.report import generate_eer_report, export_error_rates
    from pyeer.plot import plot_eer_stats


    # Calculating stats for classifier A
    stats_a = get_eer_stats(Gen, Imp)
    print(stats_a.eer)

    return stats_a.eer

test(net,test_loader)
##### INSPECT FEATURES



In [ ]:


# np.save('FEATS_gcn2wayca.npy', GCNFEATS)

In [ ]:
# FEATSRES18 = np.load('FEATS_res18.npy')
# print(FEATSRES18.shape)
# FEATSRES18 = np.squeeze(FEATSRES18);
# print(FEATSRES18.shape)
# # # NewFEATS = np.dstack((FEATS,FEATSRES18))
# # FEATS = np.concatenate((FEATS,FEATSRES18), axis=1)
# # print(NewFEATS.shape)


In [ ]:
from numpy import dot
from numpy.linalg import norm

def cossim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

pred_scores = []
gt_label = []

for i in tqdm.tqdm(range(3000)):
    for j in range(i+1,3000):
        # pred_scores.append(final[i,j].detach().cpu().numpy())
        a = cossim(GCNFEATS[i,:],GCNFEATS[j,:])
#         b = cossim(FEATSRES18[i,:],FEATSRES18[j,:])
#         pred_scores.append((1.8*a+0.2*b)/2)
        pred_scores.append(a)
#         pred_scores.append(b)
        gt_label.append(i//6 == j//6)
        

# for i in tqdm.tqdm(range(600)):
#         # pred_scores.append(final[i,j].detach().cpu().numpy())
#         pred_scores.append(cossim(FEATS[i,:],FEATS[]))
#         # gt_label.append(i//12 == j//12)
pred_scores = np.array(pred_scores)
gt_label = np.array(gt_label)

In [ ]:

Gen = pred_scores[gt_label]
Imp = pred_scores[gt_label==False]
Imp = Imp[np.random.permutation(len(Imp))[:len(Gen)]]


import seaborn as sns
sns.distplot(Gen,  kde=False, label='Gen')
# df =gapminder[gapminder.continent == 'Americas']
sns.distplot(Imp,  kde=False,label='Imp')
# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Life Expectancy of Two Continents')
plt.xlabel('Life Exp (years)')
plt.ylabel('Density')

from pyeer.eer_info import get_eer_stats
from pyeer.report import generate_eer_report, export_error_rates
from pyeer.plot import plot_eer_stats


# Calculating stats for classifier A
stats_a = get_eer_stats(Gen, Imp)
print(stats_a.eer)

In [ ]:
import numpy as np
import sklearn.metrics

"""
Python compute equal error rate (eer)
ONLY tested on binary classification

:param label: ground-truth label, should be a 1-d list or np.array, each element represents the ground-truth label of one sample
:param pred: model prediction, should be a 1-d list or np.array, each element represents the model prediction of one sample
:param positive_label: the class that is viewed as positive class when computing EER
:return: equal error rate (EER)
"""
def compute_eer(label, pred, positive_label=1):
    # all fpr, tpr, fnr, fnr, threshold are lists (in the format of np.array)
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred)#, positive_label
    fnr = 1 - tpr

    # the threshold of fnr == fpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]

    # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]

    # return the mean of eer from fpr and from fnr
    eer = (eer_1 + eer_2) / 2
    return eer

eer = compute_eer(gt_label, pred_scores)
print('The equal error rate is {:.3f}'.format(eer))

In [ ]:
del Gen 
del Imp
del pred_scores
del gt_label
del GCNFEATS
del GT
del test_loader